# LAB GenAI - LLMs - OpenAI Assistant

## Exercise 1

Create an assistant to answer a topic of your choosing:
 - Upload a file of your interest
 - Add Instructions to the prompt
 - Use the assistant in Playground mode

 https://platform.openai.com/playground/assistants

In [ ]:
import openai
import os
from dotenv import load_dotenv
from icecream import ic
from openai import OpenAI
from tabulate import tabulate


# Load environment variables from .env file
load_dotenv()

# get OpenAI API key
openai_api_key = os.getenv("OPENAI_API_KEY")


client = openai.OpenAI(api_key=openai_api_key)


# # List all assistants
# assistants = client.beta.assistants.list()

# # Print available assistant IDs
# for assistant in assistants.data:
#     print(f"Name: {assistant.name}, ID: {assistant.id}")


assistant_id = "asst_H1kg2cDQaRANsulU11HNVC8e"

# Create a thread for the conversation
thread = client.beta.threads.create()


# Send a message to the assistant
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Hello, what is project management?"
)

# Run the assistant on the created thread
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant_id
)

# Wait for completion (polling)
import time
while run.status not in ["completed", "failed"]:
    time.sleep(1)
    run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)

# Get the response messages
messages = client.beta.threads.messages.list(thread_id=thread.id)
for msg in messages.data:
    if msg.role == "assistant":
        # Extract the text content
        response_text = msg.content[0].text.value  # Extract the response text
        print(response_text)  # Print or store the response



# Project Manager Assistant

**Thread ID:** `thread_VQReOCA07rmdhvq1x8HEgANM`  
**Tokens Used:** `18076`

## Overview
Project management is the process of guiding a project from its beginning through its performance to its closure. It encompasses five sets of processes:

## 1. Initiating Processes
These involve:
- Clarifying the business need
- Defining high-level expectations
- Establishing resource budgets
- Identifying audiences that may play a role in the project

## 2. Planning Processes
These processes detail:
- Project scope
- Time frames
- Resources and risks
- Approaches to project communications, quality, and external purchases of goods and services

## 3. Executing Processes
This involves:
- Establishing and managing the project team
- Communicating with and managing project audiences
- Implementing project plans

## 4. Monitoring and Controlling Processes
These processes:
- Track performance
- Take necessary actions to ensure project plans are successfully implemented
- Ensure the desired results are achieved

## 5. Closing Processes
These involve:
- Ending all project activities

## Project Life Cycle Stages
These processes help support a project through the four stages of its life cycle:
1. **Starting the project**
2. **Organizing and preparing**
3. **Carrying out the work**
4. **Closing out the project**

---

### References
[1] [2] [3]


## Exercise 2

Talk to your assistant via the API

https://platform.openai.com/docs/assistants/overview

In [ ]:
import openai
import os
from dotenv import load_dotenv
from icecream import ic
from openai import OpenAI
from tabulate import tabulate


# Load environment variables from .env file
load_dotenv()

# get OpenAI API key
openai_api_key = os.getenv("OPENAI_API_KEY")


client = openai.OpenAI(api_key=openai_api_key)



assistant_id = "asst_H1kg2cDQaRANsulU11HNVC8e"

# Create a thread for the conversation
thread = client.beta.threads.create()

# Send a message to the assistant
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Hello, what is a common mistake in project management?"
)

# Run the assistant on the created thread
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant_id
)

# Wait for completion (polling)
import time
while run.status not in ["completed", "failed"]:
    time.sleep(1)
    run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)

# Get the response messages
messages = client.beta.threads.messages.list(thread_id=thread.id)
for msg in messages.data:
    if msg.role == "assistant":
        # Extract the text content
        response_text = msg.content[0].text.value  # Extract the response text
        print(response_text)  # Print or store the response


## Exercise 3

Create an assistant that will call a weather API, given the user's answer and return the proper answer.

See the documentation of the weather API here: https://open-meteo.com/en/docs

In [ ]:
import openai
import requests
import os
import time
import logging
from dotenv import load_dotenv
import json

# Load environment variables
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# OpenAI client
client = openai.OpenAI(api_key=api_key)

# Function to get weather forecast
def get_weather_forecast(latitude, longitude):
    logging.info(f"Fetching weather data for lat={latitude}, lon={longitude}")
    base_url = "https://api.open-meteo.com/v1/forecast"
    params = {
        "latitude": latitude,
        "longitude": longitude,
        "hourly": "temperature_2m"
    }
    response = requests.get(base_url, params=params)
    return response.json()

# Create or retrieve assistant
assistants = client.beta.assistants.list()
assistant_id = None

for assistant in assistants.data:
    if assistant.name == "Weather Assistant":
        assistant_id = assistant.id
        logging.info(f"Using existing assistant: {assistant_id}")
        break

if not assistant_id:
    assistant = client.beta.assistants.create(
        name="Weather Assistant",
        instructions="You provide weather forecasts when given a location. Use the function to fetch live weather data.",
        model="gpt-4o",
        tools=[{
            "type": "function",
            "function": {
                "name": "get_weather_forecast",
                "description": "Get the weather forecast for a specific location.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "latitude": {
                            "type": "number",
                            "description": "Latitude of the location"
                        },
                        "longitude": {
                            "type": "number",
                            "description": "Longitude of the location"
                        }
                    },
                    "required": ["latitude", "longitude"]
                }
            }
        }]
    )
    assistant_id = assistant.id
    logging.info(f"Created new assistant with ID: {assistant_id}")

# Create a conversation thread
thread = client.beta.threads.create()
logging.info(f"Created thread with ID: {thread.id}")

# User asks for weather
user_message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="What's the weather like in Berlin?"
)
logging.info("Sent user message: 'What's the weather like in Berlin?'")

# Run the assistant on the thread
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant_id
)
logging.info(f"Started assistant run: {run.id}")

# Wait for the assistant run to reach a terminal state or require action
while True:
    run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
    if run.status in ("completed", "requires_action"):
        logging.info(f"Assistant run status: {run.status}. Exiting wait loop.")
        break
    time.sleep(1)

# If the assistant requires action (e.g., a function call), process it
if run.status == "requires_action":
    logging.info("Assistant requires action: function execution needed.")

    # Extract function call details and prepare outputs
    tool_outputs = []
    for action in run.required_action.submit_tool_outputs.tool_calls:
        if action.function.name == "get_weather_forecast":
            try:
                args = json.loads(action.function.arguments)
                latitude = args.get("latitude")
                longitude = args.get("longitude")
                if latitude is None or longitude is None:
                    raise ValueError("Missing 'latitude' or 'longitude' in function arguments.")
            except (json.JSONDecodeError, ValueError, KeyError) as e:
                logging.error(f"Error parsing function call details for action {action.id}: {e}")
                continue

            logging.info(f"Executing get_weather_forecast for lat={latitude}, lon={longitude}")
            weather_data = get_weather_forecast(latitude, longitude)

            tool_outputs.append({
                "tool_call_id": action.id,
                "output": json.dumps(weather_data)  # Must be a JSON string
            })

    # Submit the function results back to the assistant
    client.beta.threads.runs.submit_tool_outputs(
        thread_id=thread.id,
        run_id=run.id,
        tool_outputs=tool_outputs
    )
    logging.info("Submitted tool outputs to assistant.")

    # After submitting tool outputs, wait until the run is completed
    while True:
        run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
        if run.status == "completed":
            logging.info("Assistant run completed after tool outputs submission.")
            break
        time.sleep(1)

# Optional: Give the assistant a moment to finalize processing
time.sleep(1)

# Retrieve and print the assistant's response
messages = client.beta.threads.messages.list(thread_id=thread.id)
for msg in messages.data:
    # Assumes the response message structure includes a text value
    try:
        response_text = msg.content[0].text.value
    except (IndexError, AttributeError, KeyError):
        response_text = "<No text response available>"
    logging.info(f"Assistant response: {response_text}")
    print(f"Assistant: {response_text}")


2025-02-09 16:50:13,591 - INFO - HTTP Request: GET https://api.openai.com/v1/assistants "HTTP/1.1 200 OK"
2025-02-09 16:50:13,615 - INFO - Using existing assistant: asst_3eXOUHpUAs92LzUgoYleC558
2025-02-09 16:50:13,886 - INFO - HTTP Request: POST https://api.openai.com/v1/threads "HTTP/1.1 200 OK"
2025-02-09 16:50:13,957 - INFO - Created thread with ID: thread_yAPgYm8v7W9INH7eQK9VhNCv
2025-02-09 16:50:14,308 - INFO - HTTP Request: POST https://api.openai.com/v1/threads/thread_yAPgYm8v7W9INH7eQK9VhNCv/messages "HTTP/1.1 200 OK"
2025-02-09 16:50:14,379 - INFO - Sent user message: 'What's the weather like in Berlin?'
2025-02-09 16:50:15,268 - INFO - HTTP Request: POST https://api.openai.com/v1/threads/thread_yAPgYm8v7W9INH7eQK9VhNCv/runs "HTTP/1.1 200 OK"
2025-02-09 16:50:15,270 - INFO - Started assistant run: run_mdKOlP67cgVsFxIBWkpcTMNg
2025-02-09 16:50:15,641 - INFO - HTTP Request: GET https://api.openai.com/v1/threads/thread_yAPgYm8v7W9INH7eQK9VhNCv/runs/run_mdKOlP67cgVsFxIBWkpcTMNg "

Assistant: The current temperature in Berlin is approximately 0°C. Here's an overview of the upcoming temperature:

- **Morning**: Temperatures starting around -1°C, warming up to about 2°C by midday.
- **Afternoon**: Temperatures peaking at around 3.7°C.
- **Evening**: Cooling down to around 0.9°C.

The temperature will be slightly below freezing in the early morning and will rise slightly during the day before dropping again in the evening.
Assistant: What's the weather like in Berlin?


### If you want to, there is a hint here:

OpenAI Chatbots / Assistants have a way to respond in json format. 

Explore the function calling functionality